In [1]:
import pandas as pd
import numpy as np
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import os
os.environ['R_HOME'] = '/root/miniconda3/envs/R/lib/R'

import rpy2.robjects as objects
from rpy2.robjects.packages import importr
base = importr('base')
r_pROC = importr("pROC")
base._libPaths()[0]

def df_stats(df):
    from tabulate import tabulate
    print("\n***** Shape: ", df.shape," *****\n")
    
    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()
    
    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(list_stat_val, columns=['Name', 'Null', 'Unique', 'Dtypes'])
    print(tabulate(df_stat_val, headers='keys', tablefmt='psql'))
    return df.head()

/tmp/ipykernel_198656/4163889655.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [20]:
df_ecg_parquet = pd.read_parquet('/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_05_09_2004-to-june-2022_n_1572280_with_labelbox_no_duplicates.parquet')

In [21]:
df_ecg_parquet['RestingECG_PatientDemographics_PatientID'] = df_ecg_parquet['RestingECG_PatientDemographics_PatientID'].astype(int)

In [27]:
df_lqts = pd.read_csv('data/validation LQT-AI - MGM-RT-20231002_SG final.csv')
display(df_lqts.head(n=5))
display(df_lqts.patient_id.unique().shape) 
#display df_lqts['patient_id'] in df_ecg_parquet['RestingECG_PatientDemographics_PatientID']
df_lqts['patient_id'] = df_lqts['patient_id'].astype(int)
#df_lqts_f = df_lqts.loc[~df_lqts['patient_id'].isin(df_ecg_parquet['RestingECG_PatientDemographics_PatientID'])]
## 24 patients not found because ECGs missing

#dsiplay df_ecg_parquet['RestingECG_PatientDemographics_PatientID'] in df_lqts['patient_id']
df_ecg_parquet_f = df_ecg_parquet.loc[df_ecg_parquet['RestingECG_PatientDemographics_PatientID'].isin(df_lqts['patient_id'])]
display(df_ecg_parquet_f.describe())

,patient_id,Dossier_exists_in_ecg,included,GENOTYPE (0=negative; LQT1=KCNQ1 P/LP/VUSstrong; LQT2=KCNH2 P/LP/VUSstrong; Other=P/LP variant in another LQT gene),sex,dob,ethnicity,Testing_modality,date_testing,age_testing,variant_gene,variant_cdna,variant_protein,variant_class,variant_comments,date_ecg_outpatient,age_ecg_outpatient,qt_manual_ecg_outpatient,qt_exclude,Diagnostico a revisar,Diagnosis,status,Comments,KCNH2,KCNQ1,Colonne4,Columna4,Columna5,Columna6
0,396027,True,1,0,0.0,8/12/62,4.0,0.0,1/21/11,48.58,0,c.3308C>A,p.Ser1103Tyr,MODIF.,Cat 3 SCN5A c.3308C>A,NaN,NaN,NaN,NaN,NaN,"Screening fam MS, QT limite",Cas Index,NaN,1.0,1.0,NaN,00559,score LQTS=2.5,NaN
1,10715,True,1,0,1.0,4/21/59,1.0,0.0,7/24/19,60.42,0,c.12919C>T,NaN,VUS,VUS RYR2,3/12/14,NaN,NaN,NaN,NaN,Mort subite inexpliqué (mais prolapsus mitral),Cas Index,NaN,1.0,1.0,NaN,000A1,NaN,NaN
2,49469,True,1,0,0.0,6/8/60,1.0,0.0,9/27/18,58.46,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FV Idiopathique,Cas Index,NaN,1.0,1.0,NaN,02782,Définitif,NaN
3,110363,True,1,0,1.0,7/26/57,1.0,1.0,11/5/09,52.42,0,NaN,p.Ile19Ser,VUS,Cat 3 KCNH2 I19S,NaN,NaN,NaN,NaN,NaN,Syndrome du QT long,Membre Famille,NaN,1.0,NaN,NaN,00021,Définitif,Définitif
4,116147,True,1,0,1.0,2/12/59,1.0,0.0,1/1/13,54.03,0,NaN,NaN,NaN,NaN,9/14/09,NaN,NaN,NaN,NaN,Autre diagnostic,Cas Index,NaN,1.0,1.0,NaN,00110,score LQTS=4 mais peu probable,NaN


(452,)

,RestingECG_PatientDemographics_PatientID,Unnamed: 0.1,RestingECG_PatientDemographics_PatientAge,RestingECG_TestDemographics_Site,RestingECG_TestDemographics_Location,RestingECG_TestDemographics_CartNumber,RestingECG_TestDemographics_AnalysisSoftwareVersion,RestingECG_TestDemographics_OverreaderID,RestingECG_TestDemographics_AcquisitionTechID,RestingECG_TestDemographics_EditorID,RestingECG_TestDemographics_HISStatus,RestingECG_RestingECGMeasurements_VentricularRate,RestingECG_RestingECGMeasurements_QRSDuration,RestingECG_RestingECGMeasurements_QTInterval,RestingECG_RestingECGMeasurements_QTCorrected,RestingECG_RestingECGMeasurements_RAxis,RestingECG_RestingECGMeasurements_TAxis,RestingECG_RestingECGMeasurements_QRSCount,RestingECG_RestingECGMeasurements_QOnset,RestingECG_RestingECGMeasurements_QOffset,RestingECG_RestingECGMeasurements_TOffset,RestingECG_RestingECGMeasurements_ECGSampleBase,RestingECG_RestingECGMeasurements_ECGSampleExponent,RestingECG_RestingECGMeasurements_QTcFrederica,RestingECG_OriginalRestingECGMeasurements_VentricularRate,RestingECG_OriginalRestingECGMeasurements_QRSDuration,RestingECG_OriginalRestingECGMeasurements_QTInterval,RestingECG_OriginalRestingECGMeasurements_QTCorrected,RestingECG_OriginalRestingECGMeasurements_RAxis,RestingECG_OriginalRestingECGMeasurements_TAxis,RestingECG_OriginalRestingECGMeasurements_QRSCount,RestingECG_OriginalRestingECGMeasurements_QOnset,RestingECG_OriginalRestingECGMeasurements_QOffset,RestingECG_OriginalRestingECGMeasurements_TOffset,RestingECG_OriginalRestingECGMeasurements_ECGSampleBase,RestingECG_OriginalRestingECGMeasurements_ECGSampleExponent,RestingECG_OriginalRestingECGMeasurements_QTcFrederica,RestingECG_QRSTimesTypes_QRS_0_Number,RestingECG_QRSTimesTypes_QRS_0_Type,RestingECG_QRSTimesTypes_QRS_0_Time,RestingECG_QRSTimesTypes_QRS_1_Number,RestingECG_QRSTimesTypes_QRS_1_Type,RestingECG_QRSTimesTypes_QRS_1_Time,RestingECG_QRSTimesTypes_QRS_2_Number,RestingECG_QRSTimesTypes_QRS_2_Type,RestingECG_QRSTimesTypes_QRS_2_Time,RestingECG_QRSTimesTypes_QRS_3_Number,RestingECG_QRSTimesTypes_QRS_3_Type,RestingECG_QRSTimesTypes_QRS_3_Time,RestingECG_QRSTimesTypes_QRS_4_Number,RestingECG_QRSTimesTypes_QRS_4_Type,RestingECG_QRSTimesTypes_QRS_4_Time,RestingECG_QRSTimesTypes_QRS_5_Number,RestingECG_QRSTimesTypes_QRS_5_Type,RestingECG_QRSTimesTypes_QRS_5_Time,RestingECG_QRSTimesTypes_QRS_6_Number,RestingECG_QRSTimesTypes_QRS_6_Type,RestingECG_QRSTimesTypes_QRS_6_Time,RestingECG_QRSTimesTypes_QRS_7_Number,RestingECG_QRSTimesTypes_QRS_7_Type,RestingECG_QRSTimesTypes_QRS_7_Time,RestingECG_QRSTimesTypes_QRS_8_Number,RestingECG_QRSTimesTypes_QRS_8_Type,RestingECG_QRSTimesTypes_QRS_8_Time,RestingECG_QRSTimesTypes_QRS_9_Number,RestingECG_QRSTimesTypes_QRS_9_Type,RestingECG_QRSTimesTypes_QRS_9_Time,RestingECG_QRSTimesTypes_QRS_10_Number,RestingECG_QRSTimesTypes_QRS_10_Type,RestingECG_QRSTimesTypes_QRS_10_Time,RestingECG_QRSTimesTypes_QRS_11_Number,RestingECG_QRSTimesTypes_QRS_11_Type,RestingECG_QRSTimesTypes_QRS_11_Time,RestingECG_QRSTimesTypes_QRS_12_Number,RestingECG_QRSTimesTypes_QRS_12_Type,RestingECG_QRSTimesTypes_QRS_12_Time,RestingECG_QRSTimesTypes_QRS_13_Number,RestingECG_QRSTimesTypes_QRS_13_Type,RestingECG_QRSTimesTypes_QRS_13_Time,RestingECG_QRSTimesTypes_GlobalRR,RestingECG_QRSTimesTypes_QTRGGR,RestingECG_RestingECGMeasurements_SystolicBP,RestingECG_RestingECGMeasurements_DiastolicBP,RestingECG_OriginalRestingECGMeasurements_SystolicBP,RestingECG_OriginalRestingECGMeasurements_DiastolicBP,RestingECG_RestingECGMeasurements_AtrialRate,RestingECG_RestingECGMeasurements_PRInterval,RestingECG_RestingECGMeasurements_PAxis,RestingECG_RestingECGMeasurements_POnset,RestingECG_RestingECGMeasurements_POffset,RestingECG_OriginalRestingECGMeasurements_AtrialRate,RestingECG_OriginalRestingECGMeasurements_PRInterval,RestingECG_OriginalRestingECGMeasurements_PAxis,RestingECG_OriginalRestingECGMeasurements_POnset,RestingECG_OriginalRestingECGMeasurements_POffset,RestingECG_PharmaData_PharmaRR

In [30]:
## Get filename from xml_path
df_ecg_parquet_f['xml_filename'] = df_ecg_parquet_f['xml_path'].apply(lambda x: x.split('/')[-1])
display(df_ecg_parquet_f.describe())

/tmp/ipykernel_198656/3592125209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ecg_parquet_f['xml_filename'] = df_ecg_parquet_f['xml_path'].apply(lambda x: x.split('/')[-1])


,RestingECG_PatientDemographics_PatientID,Unnamed: 0.1,RestingECG_PatientDemographics_PatientAge,RestingECG_TestDemographics_Site,RestingECG_TestDemographics_Location,RestingECG_TestDemographics_CartNumber,RestingECG_TestDemographics_AnalysisSoftwareVersion,RestingECG_TestDemographics_OverreaderID,RestingECG_TestDemographics_AcquisitionTechID,RestingECG_TestDemographics_EditorID,RestingECG_TestDemographics_HISStatus,RestingECG_RestingECGMeasurements_VentricularRate,RestingECG_RestingECGMeasurements_QRSDuration,RestingECG_RestingECGMeasurements_QTInterval,RestingECG_RestingECGMeasurements_QTCorrected,RestingECG_RestingECGMeasurements_RAxis,RestingECG_RestingECGMeasurements_TAxis,RestingECG_RestingECGMeasurements_QRSCount,RestingECG_RestingECGMeasurements_QOnset,RestingECG_RestingECGMeasurements_QOffset,RestingECG_RestingECGMeasurements_TOffset,RestingECG_RestingECGMeasurements_ECGSampleBase,RestingECG_RestingECGMeasurements_ECGSampleExponent,RestingECG_RestingECGMeasurements_QTcFrederica,RestingECG_OriginalRestingECGMeasurements_VentricularRate,RestingECG_OriginalRestingECGMeasurements_QRSDuration,RestingECG_OriginalRestingECGMeasurements_QTInterval,RestingECG_OriginalRestingECGMeasurements_QTCorrected,RestingECG_OriginalRestingECGMeasurements_RAxis,RestingECG_OriginalRestingECGMeasurements_TAxis,RestingECG_OriginalRestingECGMeasurements_QRSCount,RestingECG_OriginalRestingECGMeasurements_QOnset,RestingECG_OriginalRestingECGMeasurements_QOffset,RestingECG_OriginalRestingECGMeasurements_TOffset,RestingECG_OriginalRestingECGMeasurements_ECGSampleBase,RestingECG_OriginalRestingECGMeasurements_ECGSampleExponent,RestingECG_OriginalRestingECGMeasurements_QTcFrederica,RestingECG_QRSTimesTypes_QRS_0_Number,RestingECG_QRSTimesTypes_QRS_0_Type,RestingECG_QRSTimesTypes_QRS_0_Time,RestingECG_QRSTimesTypes_QRS_1_Number,RestingECG_QRSTimesTypes_QRS_1_Type,RestingECG_QRSTimesTypes_QRS_1_Time,RestingECG_QRSTimesTypes_QRS_2_Number,RestingECG_QRSTimesTypes_QRS_2_Type,RestingECG_QRSTimesTypes_QRS_2_Time,RestingECG_QRSTimesTypes_QRS_3_Number,RestingECG_QRSTimesTypes_QRS_3_Type,RestingECG_QRSTimesTypes_QRS_3_Time,RestingECG_QRSTimesTypes_QRS_4_Number,RestingECG_QRSTimesTypes_QRS_4_Type,RestingECG_QRSTimesTypes_QRS_4_Time,RestingECG_QRSTimesTypes_QRS_5_Number,RestingECG_QRSTimesTypes_QRS_5_Type,RestingECG_QRSTimesTypes_QRS_5_Time,RestingECG_QRSTimesTypes_QRS_6_Number,RestingECG_QRSTimesTypes_QRS_6_Type,RestingECG_QRSTimesTypes_QRS_6_Time,RestingECG_QRSTimesTypes_QRS_7_Number,RestingECG_QRSTimesTypes_QRS_7_Type,RestingECG_QRSTimesTypes_QRS_7_Time,RestingECG_QRSTimesTypes_QRS_8_Number,RestingECG_QRSTimesTypes_QRS_8_Type,RestingECG_QRSTimesTypes_QRS_8_Time,RestingECG_QRSTimesTypes_QRS_9_Number,RestingECG_QRSTimesTypes_QRS_9_Type,RestingECG_QRSTimesTypes_QRS_9_Time,RestingECG_QRSTimesTypes_QRS_10_Number,RestingECG_QRSTimesTypes_QRS_10_Type,RestingECG_QRSTimesTypes_QRS_10_Time,RestingECG_QRSTimesTypes_QRS_11_Number,RestingECG_QRSTimesTypes_QRS_11_Type,RestingECG_QRSTimesTypes_QRS_11_Time,RestingECG_QRSTimesTypes_QRS_12_Number,RestingECG_QRSTimesTypes_QRS_12_Type,RestingECG_QRSTimesTypes_QRS_12_Time,RestingECG_QRSTimesTypes_QRS_13_Number,RestingECG_QRSTimesTypes_QRS_13_Type,RestingECG_QRSTimesTypes_QRS_13_Time,RestingECG_QRSTimesTypes_GlobalRR,RestingECG_QRSTimesTypes_QTRGGR,RestingECG_RestingECGMeasurements_SystolicBP,RestingECG_RestingECGMeasurements_DiastolicBP,RestingECG_OriginalRestingECGMeasurements_SystolicBP,RestingECG_OriginalRestingECGMeasurements_DiastolicBP,RestingECG_RestingECGMeasurements_AtrialRate,RestingECG_RestingECGMeasurements_PRInterval,RestingECG_RestingECGMeasurements_PAxis,RestingECG_RestingECGMeasurements_POnset,RestingECG_RestingECGMeasurements_POffset,RestingECG_OriginalRestingECGMeasurements_AtrialRate,RestingECG_OriginalRestingECGMeasurements_PRInterval,RestingECG_OriginalRestingECGMeasurements_PAxis,RestingECG_OriginalRestingECGMeasurements_POnset,RestingECG_OriginalRestingECGMeasurements_POffset,RestingECG_PharmaData_PharmaRR

In [31]:

## Copy df_lqts['xml_path'] to 'data/lqts_ecg/' folder

import os
import shutil
from tqdm import tqdm

def copy_files(df, dst):
# Make dst directory if it doesn't exist

    if not os.path.exists(dst):
        os.makedirs(dst)

    for index, row in tqdm(df.iterrows()):
        file_name = row['xml_filename']
        dst_file = os.path.join(dst, file_name)
        shutil.copyfile(row['xml_path'], dst_file)

copy_files(df_ecg_parquet_f, 'data/lqts_ecg_20231016/')

0it [00:00, ?it/s]

3966it [01:58, 33.34it/s]


In [33]:
import lqtnet
#Run script "python lqtnet/extract_ecg_xml.py --source_dir data/lqts_ecg --dest_dir data/ECG_CSV" in the notebook
!python extract_ecg_xml.py --source_dir data/lqts_ecg_20231016 --dest_dir data/ECG_CSV_20231016


Saved: data/ECG_CSV_20231016/MUSE_20220921_132331_53000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221204_051536_67000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221208_193827_67000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221114_171545_27000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221204_221156_57000.csv
Saved: data/ECG_CSV_20231016/MUSE_20220902_092411_71000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221111_132755_11000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221207_004855_56000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221112_185535_15000.csv
Saved: data/ECG_CSV_20231016/MUSE_20220921_125319_19000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221207_114103_80000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221213_231444_23000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221201_013244_30000.csv
Saved: data/ECG_CSV_20231016/MUSE_20220912_202203_09000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221109_065600_70000.csv
Saved: data/ECG_CSV_20231016/MUSE_20221114_120607_93000.csv
Saved: data/ECG_CSV_20231016/MUSE_202211

In [34]:
## Get filename from xml_path
df_ecg_parquet_f['xml_filename'] = df_ecg_parquet_f['xml_path'].apply(lambda x: x.split('/')[-1])

df_ecg_parquet_f = df_ecg_parquet_f.rename(columns={
    'xml_path': 'xml_path',
    'RestingECG_PatientDemographics_PatientID': 'patient_id',
    'old_id_site_column': 'id_site',
    'RestingECG_TestDemographics_RoomID': 'id_site',
    'RestingECG_PatientDemographics_DateofBirth': 'dob',
    'RestingECG_PatientDemographics_PatientAge': 'age',
    'RestingECG_PatientDemographics_Gender': 'sex',
    'RestingECG_Order_OrderPlacementDate': 'date',
    'RestingECG_Order_OrderPlacementTime': 'hr',
    'RestingECG_RestingECGMeasurements_QTCorrected': 'qt_manual',
    'RestingECG_OriginalRestingECGMeasurements_QTCorrected': 'qt_manual_confirmed'
})

df_ecg_parquet_f['set'] = 'External validation'
df_ecg_parquet_f['ethnicity'] = 'Unknown'
df_ecg_parquet_f['qc'] = 'False'
## Create df_ecg_parquet_f['file'] by adding the df_ecg_parquet_f['xml_filename'] to 'data/ECG_CSV' but change filepath from xml to .CSV
df_ecg_parquet_f['file'] = 'data/ECG_CSV_20231016/' + df_ecg_parquet_f['xml_filename'].apply(lambda x: x.replace('.xml', '.csv'))


/tmp/ipykernel_198656/990557688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ecg_parquet_f['xml_filename'] = df_ecg_parquet_f['xml_path'].apply(lambda x: x.split('/')[-1])


In [36]:
## Check if df_ecg_parquet_f['file'] exists
df_ecg_parquet_f['file_exists'] = df_ecg_parquet_f['file'].apply(lambda x: os.path.exists(x))
display(df_ecg_parquet_f['file_exists'].value_counts())

True    3966
Name: file_exists, dtype: int64

In [54]:
df_lqts_rafik = pd.read_csv('data/validation LQT-AI - MGM-RT-20231002_SG final.csv')
## Check if df_LQTS['Dossier'] is in df_ecg['RestingECG_PatientDemographics_PatientID']
df_lqts_rafik['Dossier'] = df_lqts_rafik['patient_id'].astype(int)
## Convert Genotype : Negative = Control ; LQT1 = Type 1 or LQT2 = Type 2
df_lqts_rafik['Genotype'] = df_lqts_rafik['GENOTYPE (0=negative; LQT1=KCNQ1 P/LP/VUSstrong; LQT2=KCNH2 P/LP/VUSstrong; Other=P/LP variant in another LQT gene)'].apply(lambda x: 'Control' if x == 0 else x)
df_lqts_rafik['Genotype'] = df_lqts_rafik['GENOTYPE (0=negative; LQT1=KCNQ1 P/LP/VUSstrong; LQT2=KCNH2 P/LP/VUSstrong; Other=P/LP variant in another LQT gene)'].apply(lambda x: 'Type 1' if x == 'LQT1' else x)
df_lqts_rafik['Genotype'] = df_lqts_rafik['GENOTYPE (0=negative; LQT1=KCNQ1 P/LP/VUSstrong; LQT2=KCNH2 P/LP/VUSstrong; Other=P/LP variant in another LQT gene)'].apply(lambda x: 'Type 2' if x == 'LQT2' else x)
#REname column Genotype to lqts_type
df_lqts_rafik = df_lqts_rafik.rename(columns={'GENOTYPE (0=negative; LQT1=KCNQ1 P/LP/VUSstrong; LQT2=KCNH2 P/LP/VUSstrong; Other=P/LP variant in another LQT gene)': 'lqts_type'})
display(df_lqts_rafik.lqts_type.value_counts())

## Keep only LQTS1, LQT2 and Control
df_lqts_rafik = df_lqts_rafik.loc[df_lqts_rafik['lqts_type'].isin(['0', 'LQT1', 'LQT2'])]
df_lqts_rafik['lqts_type'] = df_lqts_rafik['lqts_type'].apply(lambda x: 'Control' if x == '0' else x)
display(df_lqts_rafik.lqts_type.value_counts())
df_lqts_rafik.to_csv('ecg_lqts_20231016.csv')


0                  348
LQT1                57
LQT2                33
Other (SCN5A)        3
Other (KCNJ2)        2
Other (TECRL)        1
Other (KCNE1)        1
Other (CACNA1C)      1
Name: lqts_type, dtype: int64

Control    348
LQT1        57
LQT2        33
Name: lqts_type, dtype: int64

In [90]:
df_lqts_merged = pd.merge(df_ecg_parquet_f, df_lqts_rafik, how='left', left_on='patient_id', right_on='patient_id')
#drop Na df_lqts_merged['lqts_type']
df_lqts_merged = df_lqts_merged.dropna(subset=['lqts_type'])
display(df_lqts_merged.lqts_type.value_counts())
df_lqts_merged.to_csv('data/df_lqts_merged_with_metadata_20231016.csv')

Control    3316
LQT1        300
LQT2        191
Name: lqts_type, dtype: int64

In [61]:
df_lqts_merged = pd.read_csv('data/df_lqts_merged_with_metadata_20231016.csv')
## df_lqts_merged replace 'data' with '/media/data1/ravram/DeepECG/'
#df_lqts_merged['npy_path'] = df_lqts_merged['npy_path'].apply(lambda x: x.replace('data', '/media/data1/ravram/DeepECG/'))
## Check if df_lqts_merged['npy_path'] exists
df_lqts_merged['npy_path_exists'] = df_lqts_merged['npy_path'].apply(lambda x: os.path.exists(x))
display(df_lqts_merged['npy_path_exists'].value_counts())
## Load df_lqts_merged npy_path for each row and add the shape of the array to df_lqts_merged['npy_shape']

df_lqts_merged['npy_shape'] = df_lqts_merged['npy_path'].apply(lambda x: np.load(x).shape)
#df_lqts_merged['npy_shape'] = df_lqts_merged['npy_shape'].apply(lambda x: x[0])


True    3966
Name: npy_path_exists, dtype: int64

### Extract files

In [92]:
import lqtnet.sierraecg
import lqtnet.import_ecgs
import lqtnet.train

In [113]:
import sys
import os

import pandas as pd
import numpy as np
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



dir2 = os.path.abspath("/volume/lqts/lqtnet")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)


import lqtnet
import lqtnet.import_ecgs

# directory containing normalized CSV files
ECG_SOURCE_DIR = ''
MODEL_PATH = '/volume/lqts/models/2023.02.26-1/'

metadata = pd.read_csv('data/df_lqts_merged_with_metadata_20231016.csv')
ext_df = metadata.query('set == "External validation"')

x_ext, erronous_files = lqtnet.import_ecgs.df_import_csv_to_numpy(ext_df, from_dir=ECG_SOURCE_DIR)
y_ext = lqtnet.import_ecgs.df_to_np_labels(ext_df)
print("Loading model from {}".format(MODEL_PATH))
model = lqtnet.train._load_model(MODEL_PATH)
print("Making predictions")
# make predictions - save this output for further analysis
y_extval_pred = model.predict(x_ext)

/tmp/ipykernel_198656/2117157493.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Loading model from /volume/lqts/models/2023.02.26-1/
Making predictions
111/111 [==============================] - 2s 11ms/step


In [114]:
# if metadata['file'] is in erronous_files then remove it from metadata
display(len(metadata))

# Drop na
#metadata = metadata.dropna(subset=['lqts_type'])

3807

In [115]:
#metadata['y_true_bin'] = label_binarize(metadata['lqts_type'], classes=np.arange(n_classes))


In [116]:
display(metadata.lqts_type.value_counts())

Control    3316
LQT1        300
LQT2        191
Name: lqts_type, dtype: int64

In [117]:
display(metadata['lqts_type'].value_counts())

Control    3316
LQT1        300
LQT2        191
Name: lqts_type, dtype: int64

In [118]:
display(len(y_extval_pred))

3522

In [119]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# Remove erronous files from metadata
metadata = metadata[~metadata['file'].isin(erronous_files)]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = 3
lw = 2


# Binarize the true labels
#n_classes = len(np.unique(metadata['lqts_type']))
#metadata['y_true_bin'] = label_binarize(metadata['lqts_type'], classes=np.arange(n_classes))

# Compute multi-class AUC using One-vs-One (ovo) strategy
auc_ovo = roc_auc_score(metadata['lqts_type'], y_extval_pred, multi_class='ovo')

# Compute multi-class AUC using One-vs-Rest (ovr) strategy
auc_ovr = roc_auc_score(metadata['lqts_type'], y_extval_pred, multi_class='ovr')

In [121]:
display(y_extval_pred   )

array([[9.8381549e-01, 6.2651993e-03, 9.9193547e-03],
       [9.9954045e-01, 1.8045424e-04, 2.7915044e-04],
       [1.0000000e+00, 2.7769005e-23, 1.6791992e-25],
       ...,
       [1.0000000e+00, 8.1606377e-17, 3.2151985e-14],
       [1.0000000e+00, 1.0442241e-11, 6.1506332e-15],
       [1.5247419e-08, 1.0000000e+00, 3.1290844e-13]], dtype=float32)

In [122]:
metadata['y_pred_0'], metadata['y_pred_1'], metadata['y_pred_2'] = y_extval_pred[:,0], y_extval_pred[:,1], y_extval_pred[:,2]
metadata['y_pred'] = np.argmax(y_extval_pred, axis=1)


In [123]:
## Groupby metadata by patient_id and keep lqts_type and mean of argmax of "y_pred_1", "y_pred_2", "y_pred_3"
metadata_grouped = metadata.groupby('patient_id').agg({'lqts_type': 'first', 'y_pred_0': 'mean', 'y_pred_1': 'mean', 'y_pred_2': 'mean'})
metadata_grouped['y_pred'] = np.argmax(metadata_grouped[['y_pred_0','y_pred_1', 'y_pred_2']].values, axis=1)


# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = 3
lw = 2


# Binarize the true labels
#n_classes = len(np.unique(metadata['lqts_type']))
#metadata['y_true_bin'] = label_binarize(metadata['lqts_type'], classes=np.arange(n_classes))

# Compute multi-class AUC using One-vs-One (ovo) strategy
auc_ovo = roc_auc_score(metadata_grouped['lqts_type'], metadata_grouped[['y_pred_0','y_pred_1','y_pred_2']], multi_class='ovo')
print(auc_ovo)
# Compute multi-class AUC using One-vs-Rest (ovr) strategy
auc_ovr = roc_auc_score(metadata_grouped['lqts_type'], metadata_grouped[['y_pred_0','y_pred_1','y_pred_2']], multi_class='ovr')
print(auc_ovr)

0.7810375514118667
0.7869049655644828


In [125]:
display(metadata)

,Unnamed: 0.2,patient_id,RestingECG_TestDemographics_AcquisitionDate,RestingECG_TestDemographics_AcquisitionTime,Unnamed: 0.1,RestingECG_MuseInfo_MuseVersion,age,RestingECG_PatientDemographics_AgeUnits,dob_x,sex_x,RestingECG_TestDemographics_DataType,RestingECG_TestDemographics_Site,RestingECG_TestDemographics_SiteName,RestingECG_TestDemographics_AcquisitionDevice,RestingECG_TestDemographics_Status,RestingECG_TestDemographics_EditListStatus,RestingECG_TestDemographics_Priority,RestingECG_TestDemographics_Location,RestingECG_TestDemographics_LocationName,RestingECG_TestDemographics_CartNumber,RestingECG_TestDemographics_AcquisitionSoftwareVersion,RestingECG_TestDemographics_AnalysisSoftwareVersion,RestingECG_TestDemographics_EditTime,RestingECG_TestDemographics_EditDate,RestingECG_TestDemographics_OverreaderID,RestingECG_TestDemographics_AcquisitionTechID,RestingECG_TestDemographics_EditorID,RestingECG_TestDemographics_TestReason,RestingECG_TestDemographics_ReferringMDLastName,RestingECG_TestDemographics_ReferringMDFirstName,RestingECG_TestDemographics_OverreaderLastName,RestingECG_TestDemographics_OverreaderFirstName,RestingECG_TestDemographics_AcquisitionTechLastName,RestingECG_TestDemographics_AcquisitionTechFirstName,RestingECG_TestDemographics_EditorLastName,RestingECG_TestDemographics_EditorFirstName,RestingECG_TestDemographics_HISStatus,RestingECG_Order_ExtraADTData1,RestingECG_RestingECGMeasurements_VentricularRate,RestingECG_RestingECGMeasurements_QRSDuration,RestingECG_RestingECGMeasurements_QTInterval,qt_manual,RestingECG_RestingECGMeasurements_RAxis,RestingECG_RestingECGMeasurements_TAxis,RestingECG_RestingECGMeasurements_QRSCount,RestingECG_RestingECGMeasurements_QOnset,RestingECG_RestingECGMeasurements_QOffset,RestingECG_RestingECGMeasurements_TOffset,RestingECG_RestingECGMeasurements_ECGSampleBase,RestingECG_RestingECGMeasurements_ECGSampleExponent,RestingECG_RestingECGMeasurements_QTcFrederica,RestingECG_OriginalRestingECGMeasurements_VentricularRate,RestingECG_OriginalRestingECGMeasurements_QRSDuration,RestingECG_OriginalRestingECGMeasurements_QTInterval,qt_manual_confirmed,RestingECG_OriginalRestingECGMeasurements_RAxis,RestingECG_OriginalRestingECGMeasurements_TAxis,RestingECG_OriginalRestingECGMeasurements_QRSCount,RestingECG_OriginalRestingECGMeasurements_QOnset,RestingECG_OriginalRestingECGMeasurements_QOffset,RestingECG_OriginalRestingECGMeasurements_TOffset,RestingECG_OriginalRestingECGMeasurements_ECGSampleBase,RestingECG_OriginalRestingECGMeasurements_ECGSampleExponent,RestingECG_OriginalRestingECGMeasurements_QTcFrederica,RestingECG_ExtraQuestions_ExtraQuestion_0_Question,RestingECG_ExtraQuestions_ExtraQuestion_0_Answer,RestingECG_ExtraQuestions_ExtraQuestion_1_Question,RestingECG_ExtraQuestions_ExtraQuestion_1_Answer,RestingECG_QRSTimesTypes_QRS_0_Number,RestingECG_QRSTimesTypes_QRS_0_Type,RestingECG_QRSTimesTypes_QRS_0_Time,RestingECG_QRSTimesTypes_QRS_1_Number,RestingECG_QRSTimesTypes_QRS_1_Type,RestingECG_QRSTimesTypes_QRS_1_Time,RestingECG_QRSTimesTypes_QRS_2_Number,RestingECG_QRSTimesTypes_QRS_2_Type,RestingECG_QRSTimesTypes_QRS_2_Time,RestingECG_QRSTimesTypes_QRS_3_Number,RestingECG_QRSTimesTypes_QRS_3_Type,RestingECG_QRSTimesTypes_QRS_3_Time,RestingECG_QRSTimesTypes_QRS_4_Number,RestingECG_QRSTimesTypes_QRS_4_Type,RestingECG_QRSTimesTypes_QRS_4_Time,RestingECG_QRSTimesTypes_QRS_5_Number,RestingECG_QRSTimesTypes_QRS_5_Type,RestingECG_QRSTimesTypes_QRS_5_Time,RestingECG_QRSTimesTypes_QRS_6_Number,RestingECG_QRSTimesTypes_QRS_6_Type,RestingECG_QRSTimesTypes_QRS_6_Time,RestingECG_QRSTimesTypes_QRS_7_Number,RestingECG_QRSTimesTypes_QRS_7_Type,RestingECG_QRSTimesTypes_QRS_7_Time,RestingECG_QRSTimesTypes_QRS_8_Number,RestingECG_QRSTimesTypes_QRS_8_Type,RestingECG_QRSTimesTypes_QRS_8_Time,RestingECG_QRSTimesTypes_QRS_9_Number,RestingECG_QRSTimesTypes_QRS_9_Type,RestingECG_QRSTimesTypes_QRS_9_Time,RestingECG_QRSTimesTypes_QRS_10_Number,RestingECG_QRSTimesTypes_QRS_10_Type,RestingECG_QRSTimesTypes_QRS_1

In [21]:
import pandas as pd
import hashlib
import os

metadata = pd.read_csv('data/LQTS_predictions_20231016.csv')

# Step 1: Create a salt dictionary
salt_dict = {pid: os.urandom(16) for pid in metadata['patient_id'].unique()}

# Step 2: Update the salt_and_hash function
def salt_and_hash(patient_id):
    salt = salt_dict[patient_id]
    salted_value = str(patient_id).encode('utf-8') + salt
    hashed_value = hashlib.sha256(salted_value).hexdigest()
    return hashed_value


metadata['patient_id_hashed'] = metadata['patient_id'].apply(salt_and_hash)


#drop dob_x and dob_y
metadata = metadata.drop(columns=['dob_x', 'dob_y','patient_id','Dossier','sex_x'])
""



''

In [22]:
# Delete all unnamed columns
metadata = metadata.loc[:, ~metadata.columns.str.contains('^Unnamed')]
# Rename sex_y to sex in metadata




In [23]:
metadata.to_csv('data/LQTS_predictions_20231016_hashed.csv')

displa